<a href="https://colab.research.google.com/github/akaicomet/NJOITOpenDataCenter-Pension-Record/blob/master/NJOITOpenDataCenter_Pension_Record_ModelBuilding_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install tqdm
!pip install scikit-learn
!pip install sodapy
!pip install matplotlib
!pip install hyperopt
!pip install bayesian-optimization
!pip install tbvaccine 
!pip install MulticoreTSNE
!pip install -U -q PyDrive

    100% |████████████████████████████████| 61kB 5.5MB/s 
    100% |████████████████████████████████| 829kB 11.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 0.0.1a1 has requirement requests~=2.18.0, but you'll have requests 2.21.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


  Stored in directory: /root/.cache/pip/wheels/4b/60/a2/6eec4adc73341c1a2a653022b99ff8e1cb090684dba83be683
Successfully built bayesian-optimization
  Stored in directory: /root/.cache/pip/wheels/27/59/53/3b52ee63add3692254c30d687fa4dff4d128d0557861fb028e
Successfully built MulticoreTSNE
    100% |████████████████████████████████| 993kB 18.4MB/s 


#1. Copy python files from  GItHub repository to local

In [2]:
!git clone https://github.com/akaicomet/NJOITOpenDataCenter.git
!git clone https://github.com/akaicomet/akaicomet_Utility.git
!git clone https://github.com/akaicomet/akaicomet_Chart.git
!git clone https://github.com/akaicomet/akaicomet_ML.git

Cloning into 'NJOITOpenDataCenter'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 90 (delta 50), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (90/90), done.
Cloning into 'akaicomet_Utility'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
Cloning into 'akaicomet_Chart'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 58 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (58/58), done.
Cloning into 'akaicomet_ML'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 174 (del

#2. Run .py files in local

In [0]:
%run -i /content/akaicomet_Utility/ErrorHandling.py

%run -i /content/NJOITOpenDataCenter/YourMoneyActivePensionMembers.py 
%run -i /content/NJOITOpenDataCenter/YourMoneyRetiredPensionMembers.py
%run -i /content/NJOITOpenDataCenter/YourMoneyPensionMembersMerge.py

%run -i /content/akaicomet_Chart/X-Ray_Scan.py
%run -i /content/akaicomet_Chart/akaicomet_Chart.py

%run -i /content/akaicomet_ML/ML_Init.py
%run -i /content/akaicomet_ML/ML_Auto.py
%run -i /content/akaicomet_ML/AutoFeatureEngineering.py


#3. Import libraries

In [0]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import random
from sodapy import Socrata
#from pandas.plotting import scatter_matrix
#from datetime import datetime
import seaborn as sb
from tqdm import tqdm
import warnings
from textwrap import wrap

In [0]:
from sklearn.externals import joblib
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#4. YourMoney and the NJOIT Open Data Center 
##4.1 YourMoney Active Pension Members API

**Pension member data load from NJOIT** 

In [6]:
client = Socrata("data.nj.gov", None)
results = client.get("45bd-gwii", limit=100000000)#100000000

**Data Cleansing and Preparation for YourMoney Active Pension Members data**

In [0]:
ActivePension_df = ActivePensionDataWrangler(results)

##4.2 YourMoney Retired Pension Members API
**Pension payment member data load from NJOIT**

In [0]:
results = client.get("8up4-62p6",   limit=100000000)

**Data Cleansing and Preparation for YourMoney Retired Pension data**

In [0]:
RetiredPension_df_grped = RetiredPensionMembersDataWrangler(results)

**Merge YourMoney Active Pension Members data and YourMoney Retired Pension data**

In [0]:
ActivePension_df = PensionMembersMerge(ActivePension_df,RetiredPension_df_grped)
del results

## 5 Recheck  Correlation Variable Validation:

In [11]:
#x_cols2 = ['_20_year_status',	'all_employers_salary_amt',		'employer_freq_pensioned', 'total_months_qty',	'location_freq_pensioned',		'current_employer_salary_rollingamt', 	'service_months_qty',	'pension_freq_pensioned', 'pension_fund_name_enc', 'pension_group_name_enc' ]
x_cols2 = ['_20_year_status','all_employers_salary_amt','employer_freq_pensioned','total_months_qty','location_freq_pensioned','service_months_qty','pension_freq_pensioned','pension_fund_name_enc','pension_group_name_enc' ]
ActivePension_df[x_cols2].corr()

,_20_year_status,all_employers_salary_amt,employer_freq_pensioned,total_months_qty,location_freq_pensioned,service_months_qty,pension_freq_pensioned,pension_fund_name_enc,pension_group_name_enc
_20_year_status,1.000000,0.295118,-0.074679,0.844411,0.082618,-0.002429,0.091064,-0.238068,-0.113550
all_employers_salary_amt,0.295118,1.000000,-0.066198,0.395345,-0.030279,-0.027708,-0.439474,0.207998,0.097105
employer_freq_pensioned,-0.074679,-0.066198,1.000000,-0.065524,0.187599,0.026186,0.040389,0.099662,-0.033238
total_months_qty,0.844411,0.395345,-0.065524,1.000000,0.067168,0.000102,0.047303,-0.187378,-0.121364
location_freq_pensioned,0.082618,-0.030279,0.187599,0.067168,1.000000,0.056022,0.308507,0.047458,-0.063150
service_months_qty,-0.002429,-0.027708,0.026186,0.000102,0.056022,1.000000,0.079330,0.052583,-0.034342
pension_freq_pensioned,0.091064,-0.439474,0.040389,0.047303,0.308507,0.079330,1.000000,-0.306339,-0.308736
pension_fund_name_enc,-0.238068,0.207998,0.099662,-0.187378,0.047458,0.052583,-0.306339,1.000000,0.185796
pension_group_name_enc,-0.113550,0.097105,-0.033238,-0.121364,-0.063150,-0.034342,-0.308736,0.185796,1.000000


# 6 Load clfs Data from Google Drive
##  6.1 Join Table 

In [12]:
#Need to change 'id' everytime

downloaded = drive.CreateFile({'id':'1XRf0Zu7tYeCLRsqxxcuT9LhfyEVDhiPo'})
downloaded.GetContentFile('clfs_score612.pkl')

downloaded = drive.CreateFile({'id':'13BdyYzlrM4eD9HpUrmAign0cmLqrY7dl'})
downloaded.GetContentFile('clfs_score623.pkl')

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

In [0]:
#downloaded = drive.CreateFile({'id':'1vA1Z9Y9OkTNp83XR3WLuAyA-HemgTErA'})
#downloaded.GetContentFile('clfs612.pkl')
#clfs = joblib.load('clfs612.pkl')

In [0]:
clfs_score612 = joblib.load('clfs_score612.pkl')
clfs_score623 = joblib.load('clfs_score623.pkl')

In [0]:
df_clfs_score = pd.DataFrame.from_dict(clfs_score612,orient='index')

In [0]:
df_clfs_score2 = pd.DataFrame.from_dict(clfs_score623,orient='index')

In [0]:
df_clfs_score = pd.concat([df_clfs_score, df_clfs_score2], axis=1, join='inner')

## 6.2  Additional Field and Manupulation

In [0]:
df_clfs_score['classifier'] = [i.split('|')[0] for i  in df_clfs_score.index.tolist()]

In [0]:
df_clfs_score['std'] = df_clfs_score.std(axis=1)

In [0]:
df_clfs_score.columns = ['Training', 'Test1', 'Test2', 'Test3', 'classifier','std']

In [21]:
df_clfs_score.sort_values(['classifier','Training','std'], ascending=[True, False, True])

,Training,Test1,Test2,Test3,classifier,std
AdaBoostClassifier|PCA StandardScaler|PowerTransformer|,0.949143,0.930667,0.946000,0.922,AdaBoostClassifier,0.012826
AdaBoostClassifier|PCA MinMaxScaler|PowerTransformer|,0.948857,0.938000,0.948667,0.863,AdaBoostClassifier,0.041399
AdaBoostClassifier|Raw PowerTransformer|QuantileTransformer|,0.947143,0.937333,0.945333,0.937,AdaBoostClassifier,0.005291
AdaBoostClassifier|Raw StandardScaler|PowerTransformer|,0.946286,0.932000,0.944000,0.940,AdaBoostClassifier,0.006277
AdaBoostClassifier|Raw Normalizer|StandardScaler|,0.946286,0.947333,0.948667,0.933,AdaBoostClassifier,0.007280
AdaBoostClassifier|Raw Normalizer|PowerTransformer|,0.946000,0.938000,0.945333,0.937,AdaBoostClassifier,0.004740
AdaBoostClassifier|KPCA MinMaxScaler|QuantileTransformer|,0.945429,0.945333,0.947333,0.939,AdaBoostClassifier,0.003635
AdaBoostClassifier|Raw StandardScaler|QuantileTransformer|,0.944857,0.943333,0.946000,0.934,AdaBoostClassifier,0.005475
AdaBoostClassifier|PCA PowerTransformer|QuantileTransformer|,0.944000,0.939333,0.944667,0.938,AdaBoostClassifier,0.003328
AdaBoostClassifier|PCA StandardScaler|QuantileTransformer|,0.943429,0.942000,0.944000,0.939,AdaBoostClassifier,0.002236


In [15]:
set([i.split('|')[1].split(' ')[0] for i  in df_clfs_score.index.tolist()])

{'ICA', 'KPCA', 'MDS', 'PCA', 'Raw'}

In [16]:
set([i.split('|')[1].split(' ')[1] for i  in df_clfs_score.index.tolist()])

{'MinMaxScaler', 'Normalizer', 'PowerTransformer', 'StandardScaler'}

In [17]:
set([i.split('|')[2] for i  in df_clfs_score.index.tolist()])

{'Normalizer', 'PowerTransformer', 'QuantileTransformer', 'StandardScaler'}

## 6.3 Refine Redimensioning to Validate Model 

In [23]:
manager = Manager()
clfs3 = manager.dict()
clfs_score3 = manager.dict()

flatten = lambda l: [item for sublist in l for item in sublist]

y_train = ActivePension_df.sample(n=5000, random_state=2)['IsPensionPaid'].as_matrix()
X_train = ActivePension_df[x_cols2].sample(n=5000, random_state=2)

X_train = X_train.astype(float)

scaling_def = {'MinMaxScaler': MinMaxScaler(),
               'QuantileTransformer_u':QuantileTransformer(output_distribution='uniform'),'QuantileTransformer_n':QuantileTransformer(output_distribution='normal'),
               'StandardScalerMTST': StandardScaler(with_mean=True, with_std=True),'StandardScalerMTSF': StandardScaler(with_mean=True, with_std=False),'StandardScalerMFST': StandardScaler(with_mean=False, with_std=True),'StandardScalerMFSF': StandardScaler(with_mean=False, with_std=False),
               'Normalizer_l2': Normalizer(norm='l2'), 'Normalizer_l1': Normalizer(norm='l1'),
               'PowerTransformer_std':PowerTransformer(standardize=True),'PowerTransformer':PowerTransformer(standardize=False)
              
              }

#scaling_def = {'MinMaxScaler': MinMaxScaler()}

redemensioning_def = {'Raw':None,
                      'PCA': PCA(n_components=0.99, svd_solver='full',whiten=False),'PCAW': PCA(n_components=0.99, svd_solver='full',whiten=True),
                      'ICA_P': FastICA(n_components=len(X_train.columns), algorithm='parallel'),'ICA_D': FastICA(n_components=len(X_train.columns), algorithm='deflation'),
                      'KPCA_rbf': KernelPCA(n_components=len(X_train.columns), kernel='rbf',n_jobs=-1),'KPCA_lin': KernelPCA(n_components=len(X_train.columns), kernel='linear',n_jobs=-1),'KPCA_pol': KernelPCA(n_components=len(X_train.columns), kernel='poly',n_jobs=-1),'KPCA_sigm': KernelPCA(n_components=len(X_train.columns), kernel='sigmoid',n_jobs=-1),'KPCA_cos': KernelPCA(n_components=len(X_train.columns), kernel='cosine',n_jobs=-1),
                      'MDS_met': MDS(n_components=len(X_train.columns), metric=True, n_jobs=-1),'MDS': MDS(n_components=len(X_train.columns), metric=False, n_jobs=-1)
                     }


#redemensioning_def = {'Raw':None}

for clf in df_clfs_score.index.tolist():
  for i in clf.split(' ')[1:]:
    for j in i.split('|'):
      if j != '':
        scaling_def_temp = {s: scaling_def[s]  for s in scaling_def if j in s}
        redemensioning_def_temp = {i: redemensioning_def[i] for i in flatten([i.split(' ') for i in clf.split('|')])[1:] if i in redemensioning_def}
        for data in map(lambda x:Redimensioning2(x[1],x[0],redemensioning_def_temp), scaling3(X_train,scaling_def_temp)):
          for x in data:                  
            #print(x[1])
            #print(x[0])
            p = Process(target=MLModelBuilding, args=(clf.split('|')[0], x[1], x[0], y_train, 0.3, clfs3,[0.9,0.1],clfs_score3))
            #MLModelBuilding(clf.split('|')[0], x[1], x[0], y_train, 0.3, clfs3,[0.9,0.1],clfs_score3)
            p.start()
            p.join(180)
            p.terminate()
              

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train score: 0.9857142857142858
Test score: 0.9066666666666666
Test2 score: 0.9666666666666667
RandomForestClassifier|Raw MinMaxScaler|



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train score: 0.9871428571428571
Test score: 0.9066666666666666
Test2 score: 0.9666666666666667
RandomForestClassifier|Raw 

Train score: 0.96
Test score: 0.91
Test2 score: 0.96
GradientBoostingClassifier|Raw MinMaxScaler|

Train score: 0.96
Test score: 0.91
Test2 score: 0.96
GradientBoostingClassifier|Raw 

best estimate parameters {'C': 0.41071634636468335, 'class_weight': 'balanced', 'fit_intercept': False, 'intercept_scaling': 0.5563573218252669, 'loss': 'squared_hinge', 'max_iter': 51, 'tol': 0.252119990158549}
Train score: 0.94
Test score: 0.91
Test2 score: 0.94
LinearSVC|Raw MinMaxScaler|

best estimate parameters {'C': 0.9786815370071502, 'class_weight': 'balanced', 'fit_intercept': True, 'intercept_scaling': 0.08670086476878891, 'loss': 'squared_hinge', 'max_iter': 92, 'tol': 0.4293827532256457}


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


...

Train score: 0.9871428571428571
Test score: 0.9066666666666666
Test2 score: 0.9666666666666667
RandomForestClassifier|Raw 

Train score: 0.96
Test score: 0.91
Test2 score: 0.96
GradientBoostingClassifier|Raw 

Train score: 0.96
Test score: 0.91
Test2 score: 0.96
GradientBoostingClassifier|Raw 

best estimate parameters {'C': 0.14340106147551357, 'class_weight': 'balanced', 'fit_intercept': False, 'intercept_scaling': 0.5328693907115676, 'loss': 'hinge', 'max_iter': 80, 'tol': 0.9160536987651704}


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC|Raw 

best estimate parameters {'C': 0.22701728450713632, 'class_weight': 'balanced', 'fit_intercept': True, 'intercept_scaling': 0.4484005757556022, 'loss': 'squared_hinge', 'max_iter': 92, 'tol': 0.8055385120608737}


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC|Raw 

best estimate parameters {'learning_rate': 1.9687168550308016, 'n_estimators': 44}
AdaBoostClassifier|Raw 

best estimate parameters {'learning_rate': 1.7243623941238897, 'n_estimators': 9}
Train score: 0.9385714285714286
Test score: 0.9066666666666666
Test2 score: 0.9333333333333333
AdaBoostClassifier|Raw 



In [0]:
clfs32 = clfs3.copy()
clfs_score32 = clfs_score3.copy()

joblib.dump(clfs32, 'clfs624.pkl', compress=True)
joblib.dump(clfs_score32, 'clfs_score624.pkl', compress=True)

In [0]:
folder_id = '1wRbAVDSYm-NJsRMUqkRsreELnGaNM88R'
file = drive.CreateFile({'mimeType': 'content/clfs624.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file.SetContentFile('clfs624.pkl')
file.Upload() 
file = drive.CreateFile({'mimeType': 'content/clfs_score624.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file.SetContentFile('clfs_score624.pkl')
file.Upload() 